In [ ]:
# pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.1 MB/s 


In [ ]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
warnings.filterwarnings('ignore')
from google.colab.patches import cv2_imshow
import numpy as np
import math

In [ ]:
from google.colab import drive
drive.mount('/content/AFLW2000')

Mounted at /content/AFLW2000


In [ ]:
import scipy.io as sio
def get_ypr_from_mat(mat_path):
    mat = sio.loadmat(mat_path)
    pre_pose_params = mat['Pose_Para'][0]
    pose_params = pre_pose_params[:3]
    return pose_params

In [ ]:
def get_list_from_filenames(file_path):
    lines = [Path(f).stem for f in glob.glob(file_path)]
    return lines

# **Process One Image**

In [ ]:
faceModule = mediapipe.solutions.holistic
with faceModule.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as faces:
    image = cv2.imread('/content/AFLW2000/MyDrive/AFLW2000/image03196.jpg')
    results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    shape = image.shape 
    face = results.face_landmarks.landmark
    NoseX = results.pose_landmarks.landmark[faceModule.PoseLandmark.NOSE].x * shape[1]
    NoseY=results.pose_landmarks.landmark[faceModule.PoseLandmark.NOSE].y   *shape[0]
    LfeoX = results.pose_landmarks.landmark[faceModule.PoseLandmark.LEFT_EYE_OUTER].x *shape[1]
    LfeoY = results.pose_landmarks.landmark[faceModule.PoseLandmark.LEFT_EYE_OUTER].y* shape[0]
    dist = ((((LfeoX - NoseX )**2) + ((LfeoY - NoseY)**2) )**0.5)
    face_row = list(np.array([[   ((landmark.x *shape[1])-NoseX)/dist, 
                                          ((landmark.y*shape[0])-NoseY)/dist] for landmark in face]).flatten())
    print(face_row)

[0.9385970918204204, 1.0073855961811582, 1.0987704975575525, 0.18202058534510773, 0.7638153737142868, 0.5106309080558985, 0.6091383695879794, -0.39179075638994004, 1.0768355524818203, -0.03194377610196116, 0.9064342588192611, -0.26462858317561716, 0.3811443770267942, -0.8277821289013069, -1.0512619081961174, -0.6672351847709166, 0.18527397274541357, -1.3035851306058668, 0.16739158030484666, -1.5806104077170502, -0.21429972542955464, -2.5541902441530633, 0.9418415599658728, 1.1224632493797821, 0.90785563127269, 1.2210231561304006, 0.8435249704876535, 1.2778288199807677, 0.8695898873306784, 1.3735381328802574, 0.9220186944394124, 1.4588547300246304, 0.9589779459305061, 1.5693172039919399, 0.9556821028770983, 1.6936608926726058, 0.8462535489323886, 1.8926594580161513, 1.0097670372270011, 0.32453886125533626, 0.7381207843327727, 0.4019979516441072, -2.7538449098836204, -1.3122261047077868, -0.5347856772795313, -0.568678132181851, -0.6782037276193652, -0.525466126268369, -0.8531181642377165

In [ ]:
mat_path = os.path.join('/content/AFLW2000/MyDrive/AFLW2000/image03196.mat')
pose = get_ypr_from_mat(mat_path)
pitch = pose[0] 
yaw = pose[1] 
roll = pose[2] 
print("[ ", yaw , " , " , pitch , " , " , roll , " ] ")

[  -0.974573  ,  0.51097006  ,  -0.36945397  ] 


# **Process All Images**

In [ ]:
import scipy.io as sio
def get_468_points(Dir_Path):
  faceModule = mediapipe.solutions.holistic
  df = pd.DataFrame(columns = list(range(940)))
  filename_list = get_list_from_filenames(Dir_Path + "/*.jpg")
  X = filename_list
  
      
  with faceModule.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as faces:
    for i in range(2000):
      img_path = os.path.join(Dir_Path , X[i] + '.jpg')
      image = cv2.imread(img_path)
      results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      shape = image.shape 
      try: 
          face = results.face_landmarks.landmark
          NoseX = results.pose_landmarks.landmark[faceModule.PoseLandmark.NOSE].x * shape[1]
          NoseY=results.pose_landmarks.landmark[faceModule.PoseLandmark.NOSE].y   *shape[0]
          LfeoX = results.pose_landmarks.landmark[faceModule.PoseLandmark.LEFT_EYE_OUTER].x *shape[1]
          LfeoY = results.pose_landmarks.landmark[faceModule.PoseLandmark.LEFT_EYE_OUTER].y* shape[0]
          dist = ((((LfeoX - NoseX )**2) + ((LfeoY - NoseY)**2) )**0.5)
          face_row = list(np.array([[   ((landmark.x *shape[1])-NoseX)/dist, 
                                      ((landmark.y*shape[0])-NoseY)/dist] for landmark in face]).flatten())
          mat_path = os.path.join(Dir_Path , X[i] + '.mat')
          pose = get_ypr_from_mat(mat_path)
          pitch = pose[0] 
          yaw = pose[1]
          roll = pose[2] 
          face_row.insert(((468*2)+0),yaw)
          face_row.insert(((468*2)+1),pitch)
          face_row.insert(((468*2)+2),roll)
          face_row.insert(((468*2)+3),X[i])
          df.loc[len(df)] = face_row 

      except:
        pass
  # df = pd.DataFrame(face_row)      
  return df

In [ ]:
Data_DF = get_468_points("/content/AFLW2000/MyDrive/AFLW2000")
Data_DF

0 

1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

11 

12 

13 

14 

15 

16 

17 

18 

19 

20 

21 

22 

23 

24 

25 

26 

27 

28 

29 

30 

31 

32 

33 

34 

35 

36 

37 

38 

39 

40 

41 

42 

43 

44 

45 

46 

47 

48 

49 

50 

51 

52 

53 

54 

55 

56 

57 

58 

59 

60 

61 

62 

63 

64 

65 

66 

67 

68 

69 

70 

71 

72 

73 

74 

75 

76 

77 

78 

79 

80 

81 

82 

83 

84 

85 

86 

87 

88 

89 

90 

91 

92 

93 

94 

95 

96 

97 

98 

99 

100 

101 

102 

103 

104 

105 

106 

107 

108 

109 

110 

111 

112 

113 

114 

115 

116 

117 

118 

119 

120 

121 

122 

123 

124 

125 

126 

127 

128 

129 

130 

131 

132 

133 

134 

135 

136 

137 

138 

139 

140 

141 

142 

143 

144 

145 

146 

147 

148 

149 

150 

151 

152 

153 

154 

155 

156 

157 

158 

159 

160 

161 

162 

163 

164 

165 

166 

167 

168 

169 

170 

171 

172 

173 

174 

175 

176 

177 

178 

179 

180 

181 

182 

183 

184 



,0,1,2,3,4,5,6,7,8,9,...,930,931,932,933,934,935,936,937,938,939
0,0.938597,1.007386,1.098770,0.182021,0.763815,0.510631,0.609138,-0.391791,1.076836,-0.031944,...,0.308671,-0.800266,0.490267,-1.006096,0.482867,-1.089393,-0.974573,0.510970,-0.369454,image03196
1,0.620653,1.375399,1.325659,0.525301,0.589380,0.879985,0.836262,-0.283631,1.391070,0.246671,...,0.390860,-0.863887,0.437160,-1.010319,0.427355,-1.148040,-1.228429,-0.368714,0.177511,image03161
2,-0.200145,0.945523,-0.232793,0.611262,-0.213700,0.632172,-0.386228,-0.038489,-0.240714,0.466111,...,0.098146,-0.705439,1.153548,-1.127718,1.229906,-1.216391,-0.040907,-0.474087,0.019801,image03238
3,0.245179,0.390212,0.669888,-0.102398,0.258672,0.086786,0.374627,-0.560250,0.706987,-0.251751,...,0.174222,-0.944195,0.335404,-1.086590,0.352655,-1.176436,-0.985426,-0.379067,0.086955,image03249
4,-0.231091,0.608912,-0.168474,0.137574,-0.230430,0.238288,-0.368546,-0.476936,-0.161499,-0.028239,...,-0.037472,-0.993423,0.832600,-1.310196,0.895753,-1.368230,0.039903,-0.192543,-0.027199,image03217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818,0.096465,1.103705,-0.078571,0.729682,0.013125,0.793496,-0.274523,0.219890,-0.121227,0.594823,...,0.095882,-0.301489,0.951424,-0.702999,1.009162,-0.780140,0.206935,-0.219317,-0.202272,image00856
1819,-0.026975,0.472753,-0.188752,0.196186,-0.057657,0.292424,-0.182060,-0.095267,-0.198417,0.100473,...,0.176497,-0.290683,0.756945,-0.397687,0.822751,-0.433359,0.580349,-0.046365,-0.023616,image00847
1820,2.354778,0.784182,2.408616,0.335732,2.304485,0.496784,2.187844,-0.035539,2.404536,0.208834,...,2.261684,-0.292790,2.736815,-0.448154,2.779947,-0.496736,-0.617419,0.137163,-0.091421,image00837
1821,0.542091,0.594646,0.214526,0.240460,0.489729,0.345595,0.221053,-0.097081,0.169589,0.126626,...,0.661764,-0.457418,1.296870,-0.812498,1.355925,-0.912849,0.983607,0.236384,-0.383734,image00861


In [ ]:
len(Data_DF)

1823

# **Save Data**

In [ ]:
Data_DF.to_csv("Data_mediapipe_Last.csv",index=False)